## Demo for SDK (latest)

This example shows how to use SDK to deploy a task. The notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [get `job_source_uri`](#get-job_source_uri)
- [choose hardware config](#choose-hardware-config)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

In [1]:
import sys
sys.path.insert(0, '..')

import os
import time
import dotenv
dotenv.load_dotenv()
from swan import SwanAPI , MCSAPI

# dev_url = "https://swanhub-cali.swanchain.io"
dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))    # prod
# for prod version
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

api_key = os.getenv("MCS_API_KEY")
mcs_api = MCSAPI(api_key)


from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), swan_api.contract_info)

In [2]:
r = swan_api.contract_info
print(r)

{'client_contract_address': '0x9c5397F804f6663326151c81bBD82bb1451059E8', 'payment_contract_address': '0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816', 'rpc_url': 'https://rpc-atom-internal.swanchain.io', 'swan_token_contract_address': '0x91B25A65b295F0405552A4bbB77879ab5e38166c'}


### Show available hardware information

In [46]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['Quebec-CA'],
  'price': '1.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'description': 'Nvidia 3090 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA'],
  'price': '16.0',
  'status': 'available'},
 {'id': 74,
  'name': 'R1ae.large',
  'description': 'Nvidia 3090 · 8 vCPU · 32 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA'],
  'price': '30.0',
  'status': 'available'}]

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [47]:
cfg_name = 'C1ae.medium' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.medium', hardware.id=1, ['Quebec-CA']
The chosen hardware_id=1


### Get `job_source_uri`
to simplify the process, here we use a existing `job_source_uri` which is a hello world application, used to create task.

In [48]:

from swan.object.source_uri import GithubRepo

repo = GithubRepo("alphaflows", "tetris-docker-image", "main", os.getenv('WALLET'), hardwares[hardware_id])

repo.get_github_tree()

# Upload Directory to MCS
response = repo.generate_source_uri('swan_test', 'mar8s1', './source.json', mcs_client = mcs_api)

print(repo.source_uri)

# job_source_uri = 'https://test-api.lagrangedao.org/spaces/5117e998-c623-4837-8af9-2b7b0ce2de7f'

mar8s1: 1.12kB [15:40, 1.22B/s] 
mar8s1: 1.12kB [00:00, 9.83kB/s]                 

https://plutotest.acl.swanipfs.com/ipfs/QmRpUkGYN3S8cqWKX4ji59rnPetuqRxtqKisV2Pdjm8trU


In [49]:
job_source_uri='https://plutotest.acl.swanipfs.com/ipfs/QmPwg1ZNn2HR6WA3zTv3iLQ4qjpv8UjZad7LcRHwEhnK6n'

### Estimate Payment amount

In [56]:
duration_hour = 1 # hour
amount = contract.estimate_payment(hardware_id, duration_hour)
print(amount)

1000000000000000000


ValueError: {'code': -32000, 'message': 'insufficient funds for transfer'}

### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [51]:
import json

duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='Quebec-CA', 
    start_in=5, 
    duration=duration, 
    job_source_uri=job_source_uri,#repo.source_uri, 
    paid=contract._wei_to_swan(amount),
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

{
  "data": {
    "task": {
      "created_at": "1714072129",
      "end_at": "1714075727",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "initialized",
      "task_detail_cid": "https://plutotest.acl.swanipfs.com/ipfs/QmT3hZoAkV6477epDryynLqmGqXsH4sPdbTLKBQA9fT3gW",
      "tx_hash": null,
      "updated_at": "1714072129",
      "uuid": "8ce77a5e-b6f2-44df-9078-3e1d27391f67"
    }
  },
  "message": "Task_uuid initialized.",
  "status": "success"
}


### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [53]:
tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
print(tx_hash)

ValueError: {'code': -32000, 'message': 'insufficient funds for transfer'}

### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [ ]:
if result_validation := swan_api.validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [ ]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

### Show result

`job_real_uri` is for show the result of application you deployed.

In [ ]:
r = swan_api.get_real_url(task_uuid)
print(r)

In [ ]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)
